In [26]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [27]:
filename = 'data/Serie B/23-24.xlsx'
data = pd.read_excel(filename)
data.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,"AvgC<2,5",AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
375,I2,2024-05-10,19:30:00,FeralpiSalo,Ternana,0,1,A,0,1,...,2.78,0.75,1.90,1.95,1.91,1.98,1.98,2.02,1.91,1.91
376,I2,2024-05-10,19:30:00,Lecco,Modena,2,3,A,1,3,...,1.97,0.25,1.83,2.03,1.89,1.99,1.91,2.03,1.86,1.94
377,I2,2024-05-10,19:30:00,Reggiana,Parma,1,1,D,1,0,...,2.41,0.25,2.08,1.73,2.12,1.78,2.17,1.90,2.07,1.76
378,I2,2024-05-10,19:30:00,Spezia,Venezia,2,1,H,0,1,...,2.02,-0.25,1.95,1.90,1.97,1.91,2.00,1.95,1.94,1.87
379,I2,2024-05-10,19:30:00,Sudtirol,Palermo,0,1,A,0,0,...,1.85,0.50,1.85,2.00,1.86,2.03,1.88,2.08,1.83,1.98


In [28]:
# Supponiamo che il DataFrame si chiami df
# Aggiungere le colonne per i goal cumulativi
data['HomeGoalsCumulative'] = 0
data['AwayGoalsCumulative'] = 0

# Creare un dizionario per tenere traccia dei goal cumulativi di ogni squadra
goals_cumulative = {}

# Iterare sulle righe del DataFrame
for index, row in data.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    home_goals = row['FTHG']
    away_goals = row['FTAG']
    
    # Inizializzare il conteggio dei goal per le squadre se non già presente
    if home_team not in goals_cumulative:
        goals_cumulative[home_team] = 0
    if away_team not in goals_cumulative:
        goals_cumulative[away_team] = 0
    
    # Assegnare i goal cumulativi fino a quel momento
    data.at[index, 'HomeGoalsCumulative'] = goals_cumulative[home_team]
    data.at[index, 'AwayGoalsCumulative'] = goals_cumulative[away_team]
    
    # Aggiornare i goal cumulativi con i goal della partita attuale
    goals_cumulative[home_team] += home_goals
    goals_cumulative[away_team] += away_goals

import pandas as pd

# Supponiamo che il DataFrame si chiami df
# Aggiungere le colonne per i punti cumulativi
data['HomePointsCumulative'] = 0
data['AwayPointsCumulative'] = 0

# Creare due dizionari per tenere traccia dei punti cumulativi di ogni squadra
points_cumulative = {}

# Iterare sulle righe del DataFrame
for index, row in data.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    result = row['FTR']
    
    # Inizializzare i punti per le squadre se non già presenti
    if home_team not in points_cumulative:
        points_cumulative[home_team] = 0
    if away_team not in points_cumulative:
        points_cumulative[away_team] = 0
    
    # Assegnare i punti cumulativi fino a quel momento
    data.at[index, 'HomePointsCumulative'] = points_cumulative[home_team]
    data.at[index, 'AwayPointsCumulative'] = points_cumulative[away_team]
    
    # Aggiornare i punti cumulativi in base al risultato della partita
    if result == 'H':  # Vittoria della squadra di casa
        points_cumulative[home_team] += 3
    elif result == 'A':  # Vittoria della squadra ospite
        points_cumulative[away_team] += 3
    elif result == 'D':  # Pareggio
        points_cumulative[home_team] += 1
        points_cumulative[away_team] += 1

# Supponiamo che il DataFrame si chiami df
# Aggiungere le colonne per i goal subiti cumulativi
data['HomeGoalsConcededCumulative'] = 0
data['AwayGoalsConcededCumulative'] = 0

# Creare un dizionario per tenere traccia dei goal subiti cumulativi di ogni squadra
goals_conceded_cumulative = {}

# Iterare sulle righe del DataFrame
for index, row in data.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    home_goals = row['FTHG']  # Goal fatti dalla squadra di casa
    away_goals = row['FTAG']  # Goal fatti dalla squadra ospite
    
    # Inizializzare i goal subiti per le squadre se non già presenti
    if home_team not in goals_conceded_cumulative:
        goals_conceded_cumulative[home_team] = 0
    if away_team not in goals_conceded_cumulative:
        goals_conceded_cumulative[away_team] = 0
    
    # Assegnare i goal subiti cumulativi fino a quel momento
    data.at[index, 'HomeGoalsConcededCumulative'] = goals_conceded_cumulative[home_team]
    data.at[index, 'AwayGoalsConcededCumulative'] = goals_conceded_cumulative[away_team]
    
    # Aggiornare i goal subiti cumulativi con i goal della partita attuale
    goals_conceded_cumulative[home_team] += away_goals  # La squadra di casa subisce i goal della squadra ospite
    goals_conceded_cumulative[away_team] += home_goals  # La squadra ospite subisce i goal della squadra di casa

data['MatchGoal'] = data['FTHG'] + data['FTAG']

data.tail(35)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,MaxCAHA,AvgCAHH,AvgCAHA,HomeGoalsCumulative,AwayGoalsCumulative,HomePointsCumulative,AwayPointsCumulative,HomeGoalsConcededCumulative,AwayGoalsConcededCumulative,MatchGoal
345,I2,2024-04-27,13:00:00,Ternana,Ascoli,0,1,A,0,0,...,1.87,2.05,1.80,38,33,37,34,45,38,1
346,I2,2024-04-27,15:15:00,Cittadella,FeralpiSalo,1,1,D,1,0,...,1.92,1.98,1.83,37,40,44,31,40,59,2
347,I2,2024-04-27,15:15:00,Cosenza,Bari,4,1,H,2,1,...,2.05,1.88,1.94,39,33,39,36,38,43,5
348,I2,2024-04-27,15:15:00,Palermo,Reggiana,1,2,A,1,0,...,1.98,1.90,1.91,58,34,52,40,48,42,3
349,I2,2024-04-27,15:15:00,Sampdoria,Como,1,1,D,0,0,...,1.88,2.04,1.81,47,54,47,67,48,37,2
350,I2,2024-05-01,11:30:00,Cremonese,Pisa,2,1,H,1,0,...,2.14,1.82,1.99,44,47,60,45,30,48,3
351,I2,2024-05-01,14:00:00,Ascoli,Cosenza,0,1,A,0,1,...,1.90,1.96,1.85,34,43,37,42,38,39,1
352,I2,2024-05-01,14:00:00,Catanzaro,Venezia,3,2,H,1,1,...,1.96,1.92,1.89,55,64,57,67,44,40,5
353,I2,2024-05-01,14:00:00,Como,Cittadella,2,1,H,0,0,...,2.08,1.79,2.02,55,38,68,45,38,41,3
354,I2,2024-05-01,14:00:00,Spezia,Palermo,1,0,H,1,0,...,1.94,1.92,1.88,31,59,37,52,46,50,1


In [29]:
resultFileName = filename[:-5] + '-E.xlsx'
data.to_excel(resultFileName)

## TODO
- Average Goal Per Match H/A
